In [1]:
import duckdb
con = duckdb.execute(
    "IMPORT DATABASE '../data/db'"
)
df = con.execute("SELECT * FROM crew_members_by_date").df()
con.close()
df

,hour_start,ship_id,role,distinct_crew
0,2025-05-01 17:00:00-05:00,S1,bartender,1
1,2025-05-01 17:00:00-05:00,S1,server,1
2,2025-05-01 17:00:00-05:00,S2,bartender,1
3,2025-05-01 17:00:00-05:00,S2,server,1
4,2025-05-01 18:00:00-05:00,S1,bartender,1
...,...,...,...,...
28083,2025-07-25 16:00:00-05:00,S1,housekeeper,1
28084,2025-07-25 16:00:00-05:00,S1,security,1
28085,2025-07-25 17:00:00-05:00,S1,cook,1
28086,2025-07-25 17:00:00-05:00,S1,housekeeper,1


In [4]:
import plotly.express as px

import plotly.express as px
import pandas as pd

# Ensure datetime type
df["hour_start"] = pd.to_datetime(df["hour_start"])

# Sort data
dff = df.sort_values(["ship_id", "role", "hour_start"])

# Plot: each role gets its own row
fig = px.line(
    dff,
    x="hour_start",
    y="distinct_crew",
    color="ship_id",           # color by ship now, since role is separated
    facet_row="role",          # 👈 put roles in separate rows
    facet_col_wrap=3,          # optional if you have many ships, remove if not needed
    title="Distinct Crew Over Time by Ship and Role",
    labels={"hour_start": "Time", "distinct_crew": "Distinct Crew", "role": "Role", "ship_id": "Ship"},
    template="plotly_dark",
)

fig.update_layout(legend_title_text="Ship", hovermode="x unified")
fig.update_xaxes(matches=None, showgrid=True)
fig.update_yaxes(showgrid=True)
fig.update_xaxes(matches='x', showticklabels=True)  # share x-axis
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))  # clean row titles
fig.update_layout(height=250 * dff["role"].nunique())  # 250px per row



fig.show()

In [5]:
con = duckdb.execute(
    "IMPORT DATABASE '../data/db'"
)
df2 = con.execute("SELECT * FROM crew_members_by_dow").df()
con.close()
df2

,dow,hour,ship_id,role,avg_crew,min_crew,max_crew
0,0,0,S1,bartender,1.0,1,1
1,0,0,S1,server,1.0,1,1
2,0,0,S1,security,1.0,1,1
3,0,0,S1,cook,1.0,1,1
4,0,0,S1,housekeeper,1.0,1,1
...,...,...,...,...,...,...,...
2515,6,23,S3,housekeeper,1.0,1,1
2516,6,23,S3,security,1.0,1,1
2517,6,23,S3,bartender,1.0,1,1
2518,6,23,S3,server,1.0,1,1


In [7]:
df2.describe()

,dow,hour,avg_crew,min_crew,max_crew
count,2520.000000,2520.00000,2520.000000,2520.0,2520.000000
mean,3.000000,11.50000,1.045299,1.0,1.123810
std,2.000397,6.92356,0.124451,0.0,0.329429
min,0.000000,0.00000,1.000000,1.0,1.000000
25%,1.000000,5.75000,1.000000,1.0,1.000000
50%,3.000000,11.50000,1.000000,1.0,1.000000
75%,5.000000,17.25000,1.000000,1.0,1.000000
max,6.000000,23.00000,1.500000,1.0,2.000000


In [9]:
# Barplot of average crew by Day-of-Week and Hour, faceted by Ship (rows) and Role (columns)

# Prepare day-of-week labels
dow_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
df2["dow_name"] = df2["dow"].map(dow_map)
df2["dow_name"] = pd.Categorical(df2["dow_name"], categories=dow_order, ordered=True)

# Build a combined categorical for x-axis: "DOW HH"
df2["dow_hour"] = df2.apply(lambda r: f"{r['dow_name']} {int(r['hour']):02d}", axis=1)
# Order x-axis: all hours 00..23 grouped per DOW in calendar order
dow_hour_order = [f"{d} {h:02d}" for d in dow_order for h in range(24)]
df2["dow_hour"] = pd.Categorical(df2["dow_hour"], categories=dow_hour_order, ordered=True)

# Single barplot figure: facets by ship and role
fig_bar = px.bar(
    df2,
    x="dow_hour",
    y="avg_crew",
    facet_row="ship_id",
    facet_col="role",
    category_orders={"dow_hour": dow_hour_order},
    labels={"dow_hour": "Day of Week and Hour", "avg_crew": "Average Crew", "ship_id": "Ship", "role": "Role"},
    title="Average Crew by Day-of-Week and Hour, Faceted by Ship and Role",
    template="plotly_dark",
)

# Styling
fig_bar.update_layout(
    hovermode="x unified",
    bargap=0.1,
    legend_title_text="",
    height=max(400, 280 * df2["ship_id"].nunique()),
    width=max(900, 260 * df2["role"].nunique()),
)
fig_bar.update_xaxes(tickangle=-45, showgrid=True)
fig_bar.update_yaxes(showgrid=True)

# Clean facet labels
fig_bar.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig_bar.show()


In [10]:
# Single barplot figure: facets by ship and role
fig_bar = px.bar(
    df2,
    x="dow_hour",
    y="max_crew",
    facet_row="ship_id",
    facet_col="role",
    category_orders={"dow_hour": dow_hour_order},
    labels={"dow_hour": "Day of Week and Hour", "max_crew": "Max Crew", "ship_id": "Ship", "role": "Role"},
    title="Average Crew by Day-of-Week and Hour, Faceted by Ship and Role",
    template="plotly_dark",
)

# Styling
fig_bar.update_layout(
    hovermode="x unified",
    bargap=0.1,
    legend_title_text="",
    height=max(400, 280 * df2["ship_id"].nunique()),
    width=max(900, 260 * df2["role"].nunique()),
)
fig_bar.update_xaxes(tickangle=-45, showgrid=True)
fig_bar.update_yaxes(showgrid=True)

# Clean facet labels
fig_bar.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig_bar.show()


In [11]:
# Single barplot figure: facets by ship and role
fig_bar = px.bar(
    df2,
    x="dow_hour",
    y="min_crew",
    facet_row="ship_id",
    facet_col="role",
    category_orders={"dow_hour": dow_hour_order},
    labels={"dow_hour": "Day of Week and Hour", "min_crew": "Min Crew", "ship_id": "Ship", "role": "Role"},
    title="Average Crew by Day-of-Week and Hour, Faceted by Ship and Role",
    template="plotly_dark",
)

# Styling
fig_bar.update_layout(
    hovermode="x unified",
    bargap=0.1,
    legend_title_text="",
    height=max(400, 280 * df2["ship_id"].nunique()),
    width=max(900, 260 * df2["role"].nunique()),
)
fig_bar.update_xaxes(tickangle=-45, showgrid=True)
fig_bar.update_yaxes(showgrid=True)

# Clean facet labels
fig_bar.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig_bar.show()
